In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

import pandas as pd
import numpy as np
from modules import data_filler, preprocessing

from currency_converter import CurrencyConverter, RateNotFoundError
from forex_python.converter import CurrencyRates, RatesNotAvailableError
import datetime


DATA_FOLDER = "../data"

In [2]:
movies = pd.read_csv(f'{DATA_FOLDER}/final_dataset_v2.csv').convert_dtypes()
movies['date'] = pd.to_datetime(movies['date'])
movies.head()

,id,description,keywords,collection,production_countries,popularity,average_vote,num_votes,language,imdb_id,...,title,director,production_companies,cast,genres,revenue,budget,description2,tagline,movie_classification
0,2,Taisto Kasurinen is a Finnish coal miner whose...,"underdog, prison, factory worker, prisoner, he...",<NA>,Finland,3.860491,7.1,44,fi,tt0094675,...,Ariel,Aki Kaurismäki,"Villealfa Filmproduction Oy, Finnish Film Foun...","Turo Pajala, Susanna Haavisto, Matti Pellonpää...","drama, crime",0,0,A Finnish man goes to the city to find a job a...,<NA>,unclassified
1,3,"An episode in the life of Nikander, a garbage ...","salesclerk, helsinki, garbage, independent film",<NA>,Finland,2.292110,7.1,35,fi,tt0092149,...,Shadows in Paradise,Aki Kaurismäki,Villealfa Filmproduction Oy,"Matti Pellonpää, Kati Outinen, Sakari Kuosmane...","drama, comedy",0,0,"An episode in the life of Nikander, a garbage ...",<NA>,unclassified
2,5,It's Ted the Bellhop's first night on the job....,"hotel, new year's eve, witch, bet, hotel room,...",<NA>,United States of America,9.026586,6.5,539,en,tt0113101,...,Four Rooms,"Allison Anders, Alexandre Rockwell, Robert Rod...","Miramax Films, A Band Apart","Tim Roth, Antonio Banderas, Jennifer Beals, Ma...","crime, comedy",4257354,4000000,Four interlocking tales that take place in a f...,Twelve outrageous guests. Four scandalous requ...,minor success
3,6,"While racing to a boxing match, Frank, Mike, J...","chicago, drug dealer, boxing match, escape, on...",<NA>,"Japan, United States of America",5.538671,6.4,79,en,tt0107286,...,Judgment Night,Stephen Hopkins,"Universal Pictures, Largo Entertainment, JVC E...","Emilio Estevez, Cuba Gooding Jr., Denis Leary,...","action, thriller, crime",12526677,21000000,Four friends on their way to a boxing match ge...,Don't move. Don't whisper. Don't even breathe.,flop
4,11,Princess Leia is captured and held hostage by ...,"android, galaxy, hermit, death star, lightsabe...",Star Wars Collection,United States of America,42.149697,8.1,6778,en,tt0076759,...,Star Wars: Episode IV - A New Hope,George Lucas,"Lucasfilm, Twentieth Century Fox Film Corporation","Mark Hamill, Harrison Ford, Carrie Fisher, Pet...","adventure, action, science fiction",775512064,11000000,Luke Skywalker joins forces with a Jedi Knight...,"A long time ago in a galaxy far, far away...",super hit


In [3]:
preprocessing.print_stats(movies)

Lenght: 45411 Available budget: 10116 Available revenue: 20259
Missing values (NaN): 

 id                          0
description               529
keywords                14325
collection              40107
production_countries     4141
popularity                  3
average_vote                3
num_votes                   3
language                    0
imdb_id                     0
poster_url                380
runtime                     1
date                       29
title                       0
director                  119
production_companies     6954
cast                      652
genres                      9
revenue                     0
budget                      0
description2              709
tagline                 23304
movie_classification        0
dtype: int64

Usable rows according to target: 8139


In [4]:
movies.dtypes

id                               Int64
description                     string
keywords                        string
collection                      string
production_countries            string
popularity                     float64
average_vote                   float64
num_votes                        Int64
language                        string
imdb_id                         string
poster_url                      string
runtime                          Int64
date                    datetime64[ns]
title                           string
director                        string
production_companies            string
cast                            string
genres                          string
revenue                          Int64
budget                           Int64
description2                    string
tagline                         string
movie_classification            string
dtype: object

## First dataset

The first dataset URL: https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset?select=IMDb+movies.csv

In [5]:
movies_1 = pd.read_csv(f'{DATA_FOLDER}/IMDb movies.csv', parse_dates=['date_published']).convert_dtypes()
movies_1.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,<NA>,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,537,$ 2250,<NA>,<NA>,<NA>,7,7
1,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",<NA>,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.9,171,<NA>,<NA>,<NA>,<NA>,4,2
2,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,420,$ 45000,<NA>,<NA>,<NA>,24,3
3,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2019,<NA>,<NA>,<NA>,<NA>,28,14
4,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...","From the Manger to the Cross; or, Jesus of Naz...",1912,1913-01-01,"Biography, Drama",60,USA,English,Sidney Olcott,...,"R. Henderson Bland, Percy Dyer, Gene Gauntier,...","An account of the life of Jesus Christ, based ...",5.7,438,<NA>,<NA>,<NA>,<NA>,12,5


In [6]:
movies_1.columns

Index(['imdb_title_id', 'title', 'original_title', 'year', 'date_published',
       'genre', 'duration', 'country', 'language', 'director', 'writer',
       'production_company', 'actors', 'description', 'avg_vote', 'votes',
       'budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics'],
      dtype='object')

In [7]:
# first convert dataframe with the same format
movies_1 = movies_1.drop(['year', 'usa_gross_income', 'original_title', 
                          'writer', 'avg_vote', 'votes', 'metascore',
                          'reviews_from_users', 'reviews_from_critics'], 1)
movies_1 = movies_1.rename(columns={'imdb_title_id': 'imdb_id', 'date_published': 'date', 
                                    'genre': 'genres', 'country': 'production_countries',
                                    'production_company': 'production_companies', 
                                    'actors': 'cast', 'worlwide_gross_income': 'revenue'})

In [8]:
preprocessing.print_stats(movies_1)

Lenght: 81273 Available budget: 81273 Available revenue: 81273
Missing values (NaN): 

 imdb_id                     0
title                       0
date                        0
genres                      0
duration                    0
production_countries       39
language                  755
director                   73
production_companies     4325
cast                       66
description              2430
budget                  58469
revenue                 51381
dtype: int64


In [9]:
# get movies with unavailable revenue or budget
unavailable_budget_revenue = movies[(movies.budget == 0) | (movies.revenue == 0)][['imdb_id', 'title']]

In [10]:
movies_1[(movies_1.imdb_id.isin(unavailable_budget_revenue.imdb_id)) & \
         ((~movies_1.budget.isna()) | (~movies_1.budget.isna()))]

,imdb_id,title,date,genres,duration,production_countries,language,director,production_companies,cast,description,budget,revenue
0,tt0000574,The Story of the Kelly Gang,1906-12-26,"Biography, Crime, Drama",70,Australia,<NA>,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,$ 2250,<NA>
2,tt0002101,Cleopatra,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,$ 45000,<NA>
8,tt0002461,Richard III,1912-10-15,Drama,55,"France, USA",English,"André Calmettes, James Keane",Le Film d'Art,"Robert Gemp, Frederick Warde, Albert Gardner, ...",Richard of Gloucester uses manipulation and mu...,$ 30000,<NA>
22,tt0003740,Cabiria,1914-01-01,"Adventure, Drama, History",148,Italy,Italian,Giovanni Pastrone,Itala Film,"Carolina Catena, Lidia Quaranta, Gina Marangon...",Three centuries before Christus. Young Cabiria...,$ 210000,<NA>
33,tt0004181,Judith of Bethulia,1914-03-08,Drama,61,USA,English,D.W. Griffith,Biograph Company,"Blanche Sweet, Henry B. Walthall, Mae Marsh, R...",A religious woman seeks to save her people fro...,$ 40000,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76866,tt6194850,Zero 3,2017-01-27,Thriller,88,Lithuania,Lithuanian,Emilis Velyvis,Kinokultas,"Andrius Bialobzeskis, Ramunas Cicenas, Arvydas...",A outrageous crime - action - comedy examining...,EUR 350000,$ 1142638
77236,tt6334884,Hickok,2017-07-07,Western,88,USA,English,Timothy Woodward Jr.,Status Media & Entertainment,"Luke Hemsworth, Kris Kristofferson, Trace Adki...","Legendary lawman and gunslinger, Wild Bill Hic...",$ 8000000,<NA>
77340,tt6389344,Tri bogatyrya i Morskoy tsar,2017-01-01,"Animation, Action, Adventure",75,Russia,Russian,Konstantin Feoktistov,Melnitsa Animation Studio,"Aleksandr Boyarskiy, Dmitriy Bykovskiy-Romasho...",<NA>,$ 4000000,$ 13816357
77665,tt6569546,Vurdalaki,2017-02-22,"Action, Fantasy, Horror",90,Russia,Russian,Sergey Ginzburg,Gorad,"Lera Arefyeva, Anna Arlanova, Yuliya Aug, Anat...",In the distant part of the Carpathian Mountain...,RUR 162000000,$ 782438


We have around 6000 rows for this dataset either budget or revenue is not NaN, which we need.

There are two problems for this dataset:
* First, the budget and revenue is also given with different currencies like EUR, FR etc.
* The contry names is not matching with the original dataset like USA, UK etc.

For the second issue:

In [11]:
original_countries = preprocessing.list_column_to_long_format(movies, "production_countries")["production_countries"].unique()
imdb_countries = preprocessing.list_column_to_long_format(movies_1, "production_countries")["production_countries"].unique()

different_countries1 = np.setdiff1d(original_countries, imdb_countries)
different_countries2 = np.setdiff1d(imdb_countries, original_countries)

In [12]:
different_countries1

array(['Anguilla', 'Antarctica', 'Barbados', 'Brunei Darussalam', 'Congo',
       "Cote D'Ivoire", 'Faeroe Islands', 'French Polynesia',
       'French Southern Territories', 'Kyrgyz Republic',
       "Lao People's Democratic Republic", 'Libyan Arab Jamahiriya',
       'Macedonia', 'Madagascar', 'Palestinian Territory',
       'Syrian Arab Republic', 'Timor-Leste', 'United Kingdom',
       'United States Minor Outlying Islands', 'United States of America'],
      dtype=object)

In [13]:
different_countries2

array(['Andorra', 'Bahrain', 'Belize', 'British Virgin Islands', 'Brunei',
       'Burma', 'Cape Verde', "Côte d'Ivoire", 'Djibouti',
       'Faroe Islands', 'Federal Republic of Yugoslavia', 'Fiji',
       'Guadeloupe', 'Guinea-Bissau', 'Holy See (Vatican City State)',
       'Isle Of Man', 'Korea', 'Kyrgyzstan', 'Laos', 'Lesotho', 'Libya',
       'Malawi', 'Maldives', 'Mozambique', 'New Caledonia', 'Niger',
       'North Vietnam', 'Oman', 'Palestine', 'Republic of Macedonia',
       'Reunion', 'Sudan', 'Suriname', 'Svalbard And Jan Mayen',
       'Swaziland', 'Syria', 'The Democratic Republic Of Congo', 'UK',
       'USA', 'Vanuatu', 'West Germany', 'Yemen', 'Zaire', 'Zambia'],
      dtype=object)

we can match the countries between these two symetrical differences above.

In [14]:
convension_dict = {'USA': 'United States of America', 'UK':'United Kingdom', 'Syria':'Syrian Arab Republic',
                  'West Germany':'Germany', 'Palestine':'Palestinian Territory', 
                  'Laos':"Lao People's Democratic Republic", 'Libya':"Libyan Arab Jamahiriya",
                  'The Democratic Republic Of Congo':"Congo", 'Faroe Islands':"Faeroe Islands",
                  'Republic of Macedonia':"Macedonia", 'British Virgin Islands':"Anguilla",
                  "Côte d'Ivoire":"Cote D'Ivoire", 'Brunei':"Brunei Darussalam", 'Kyrgyzstan':"Kyrgyz Republic",
                  'Niger': 'Nigeria', 'North Vietnam': 'Vietnam', 'Federal Republic of Yugoslavia': 'Yugoslavia',
                  }
for key, value in convension_dict.items():
    movies_1['production_countries'] = movies_1['production_countries'].str.replace(key, value)

In [15]:
# check after the operations if the problem still persists
original_countries = preprocessing.list_column_to_long_format(movies, "production_countries")["production_countries"].unique()
imdb_countries = preprocessing.list_column_to_long_format(movies_1, "production_countries")["production_countries"].unique()

different_countries1 = np.setdiff1d(original_countries, imdb_countries)
different_countries2 = np.setdiff1d(imdb_countries, original_countries)

In [16]:
different_countries1

array(['Antarctica', 'Barbados', 'French Polynesia',
       'French Southern Territories', 'Madagascar', 'Timor-Leste',
       'United States Minor Outlying Islands'], dtype=object)

In [17]:
different_countries2

array(['Andorra', 'Bahrain', 'Belize', 'Burma', 'Cape Verde', 'Djibouti',
       'Fiji', 'Guadeloupe', 'Guinea-Bissau',
       'Holy See (Vatican City State)', 'Isle Of Man', 'Korea', 'Lesotho',
       'Malawi', 'Maldives', 'Mozambique', 'New Caledonia', 'Nigeriaia',
       'Oman', 'Reunion', 'Sudan', 'Suriname', 'Svalbard And Jan Mayen',
       'Swaziland', 'Vanuatu', 'Yemen', 'Zaire', 'Zambia'], dtype=object)

For the currency problem, we can use currency marker in budget/revenue columns and the date of the movie. By using currency marker and date of movies for each row, we can convert each currency to US dollar using historical exchange rates.

In [18]:
def convert_currency(df, column):
    c1 = CurrencyConverter(fallback_on_missing_rate=True, fallback_on_missing_rate_method='linear_interpolation')
    c2 = CurrencyRates(force_decimal=False)
    def _convert_currency(money, date, verbose=True):
        """ Convert given currency to US Dollar using two different converters"""
        marker, amount = str(money).strip().split(' ')
        if marker == '$' or marker == 'USD':
            return int(amount)

        try:
            res = c1.convert(int(amount), marker, 'USD', date=date)
        except (ValueError, RateNotFoundError) as e:
            
            if verbose:
                print(f'First converter error: {e} | money: {money}  date: {date}')
            try:
                res =  c2.convert(marker, 'USD', int(amount))
            except RatesNotAvailableError as e:
                if verbose:
                    print(f'Second converter error: {e} | money: {money}  date: {date}')
                
                return np.nan

        return int(res)
    
    return df[df[column].notnull()][[column, 'date']].apply(lambda x: _convert_currency(*x), axis=1)

    
    
converted_revenues = convert_currency(movies_1, 'revenue')
converted_budget = convert_currency(movies_1, 'budget')

First converter error: 1995-01-15 not in INR bounds 2009-01-02/2020-06-09 | money: INR 250000000  date: 1995-01-15 00:00:00
First converter error: 1994-01-14 not in INR bounds 2009-01-02/2020-06-09 | money: INR 45000000  date: 1994-01-14 00:00:00
First converter error: 1999-04-10 not in INR bounds 2009-01-02/2020-06-09 | money: INR 440000000  date: 1999-04-10 00:00:00
First converter error: 1999-11-07 not in INR bounds 2009-01-02/2020-06-09 | money: INR 500000000  date: 1999-11-07 00:00:00
First converter error: 2000-04-14 not in INR bounds 2009-01-02/2020-06-09 | money: INR 350000000  date: 2000-04-14 00:00:00
First converter error: 2000-10-27 not in INR bounds 2009-01-02/2020-06-09 | money: INR 300000000  date: 2000-10-27 00:00:00
First converter error: 1995-10-23 not in INR bounds 2009-01-02/2020-06-09 | money: INR 220000000  date: 1995-10-23 00:00:00
First converter error: 2001-11-16 not in INR bounds 2009-01-02/2020-06-09 | money: INR 33105600  date: 2001-11-16 00:00:00
First conv

Second converter error: Currency Rates Source Not Ready | money: DEM 8800000  date: 1953-10-05 00:00:00
First converter error: 1944-08-07 not in GBP bounds 1999-01-04/2020-06-09 | money: GBP 200000  date: 1944-08-07 00:00:00
First converter error: 1945-12-22 not in SEK bounds 1999-01-04/2020-06-09 | money: SEK 290000  date: 1945-12-22 00:00:00
First converter error: 1945-11-26 not in GBP bounds 1999-01-04/2020-06-09 | money: GBP 170000  date: 1945-11-26 00:00:00
First converter error: 1945-12-17 not in GBP bounds 1999-01-04/2020-06-09 | money: GBP 200000  date: 1945-12-17 00:00:00
First converter error: 1946-01-21 not in GBP bounds 1999-01-04/2020-06-09 | money: GBP 900000  date: 1946-01-21 00:00:00
First converter error: 1946-09-16 not in GBP bounds 1999-01-04/2020-06-09 | money: GBP 1278000  date: 1946-09-16 00:00:00
First converter error: 1946-12-26 not in SEK bounds 1999-01-04/2020-06-09 | money: SEK 405000  date: 1946-12-26 00:00:00
First converter error: 1947-05-03 not in GBP bou

Second converter error: Currency Rates Source Not Ready | money: NLG 420000  date: 1958-08-19 00:00:00
First converter error: 1958-08-08 not in GBP bounds 1999-01-04/2020-06-09 | money: GBP 81000  date: 1958-08-08 00:00:00
First converter error: 1958-08-01 not in GBP bounds 1999-01-04/2020-06-09 | money: GBP 400000  date: 1958-08-01 00:00:00
First converter error: NLG is not a supported currency | money: NLG 450000  date: 1958-10-24 00:00:00
Second converter error: Currency Rates Source Not Ready | money: NLG 450000  date: 1958-10-24 00:00:00
First converter error: 1959-04-24 not in GBP bounds 1999-01-04/2020-06-09 | money: GBP 70000  date: 1959-04-24 00:00:00
First converter error: DEM is not a supported currency | money: DEM 2000000  date: 1959-04-07 00:00:00
Second converter error: Currency Rates Source Not Ready | money: DEM 2000000  date: 1959-04-07 00:00:00
First converter error: FRF is not a supported currency | money: FRF 450000000  date: 1959-01-25 00:00:00
Second converter er

Second converter error: Currency Rates Source Not Ready | money: FRF 5300000  date: 1965-03-24 00:00:00
First converter error: 1965-05-01 not in GBP bounds 1999-01-04/2020-06-09 | money: GBP 25000  date: 1965-05-01 00:00:00
First converter error: 1964-09-01 not in GBP bounds 1999-01-04/2020-06-09 | money: GBP 25000  date: 1964-09-01 00:00:00
First converter error: FRF is not a supported currency | money: FRF 1350000  date: 1964-09-09 00:00:00
Second converter error: Currency Rates Source Not Ready | money: FRF 1350000  date: 1964-09-09 00:00:00
First converter error: 1965-06-18 not in GBP bounds 1999-01-04/2020-06-09 | money: GBP 140000  date: 1965-06-18 00:00:00
First converter error: 1964-10-18 not in GBP bounds 1999-01-04/2020-06-09 | money: GBP 150000  date: 1964-10-18 00:00:00
First converter error: 1964-04-23 not in GBP bounds 1999-01-04/2020-06-09 | money: GBP 30000  date: 1964-04-23 00:00:00
First converter error: 1965-01-06 not in JPY bounds 1999-01-04/2020-06-09 | money: JPY 

Second converter error: Currency Rates Source Not Ready | money: DEM 25000  date: 1978-04-13 00:00:00
First converter error: 1968-10-11 not in DKK bounds 1999-01-04/2020-06-09 | money: DKK 2920357  date: 1968-10-11 00:00:00
First converter error: 1968-10-11 not in DKK bounds 1999-01-04/2020-06-09 | money: DKK 1000000  date: 1968-10-11 00:00:00
First converter error: 1968-03-12 not in ROL bounds 1999-01-04/2005-06-30 | money: ROL 4767000  date: 1968-03-12 00:00:00
Second converter error: Currency Rates Source Not Ready | money: ROL 4767000  date: 1968-03-12 00:00:00
First converter error: ITL is not a supported currency | money: ITL 137000000  date: 1968-08-14 00:00:00
Second converter error: Currency Rates Source Not Ready | money: ITL 137000000  date: 1968-08-14 00:00:00
First converter error: 1968-09-29 not in SEK bounds 1999-01-04/2020-06-09 | money: SEK 2800000  date: 1968-09-29 00:00:00
First converter error: FIM is not a supported currency | money: FIM 900000  date: 1968-09-13 00

...

Second converter error: Currency Rates Source Not Ready | money: BDT 25000000  date: 2014-02-14 00:00:00
First converter error: BDT is not a supported currency | money: BDT 40000000  date: 2013-10-16 00:00:00
Second converter error: Currency Rates Source Not Ready | money: BDT 40000000  date: 2013-10-16 00:00:00
First converter error: BDT is not a supported currency | money: BDT 65000000  date: 2013-08-09 00:00:00
Second converter error: Currency Rates Source Not Ready | money: BDT 65000000  date: 2013-08-09 00:00:00
First converter error: IRR is not a supported currency | money: IRR 4000000000  date: 2013-03-13 00:00:00
Second converter error: Currency Rates Source Not Ready | money: IRR 4000000000  date: 2013-03-13 00:00:00
First converter error: IRR is not a supported currency | money: IRR 59000000000  date: 2011-03-17 00:00:00
Second converter error: Currency Rates Source Not Ready | money: IRR 59000000000  date: 2011-03-17 00:00:00
First converter error: TWD is not a supported cur

Second converter error: Currency Rates Source Not Ready | money: EGP 24000000  date: 2014-10-01 00:00:00
First converter error: 2007-03-09 not in INR bounds 2009-01-02/2020-06-09 | money: INR 90000000  date: 2007-03-09 00:00:00
First converter error: 2015-05-15 not in TRL bounds 1999-01-04/2004-12-31 | money: TRL 2000000  date: 2015-05-15 00:00:00
Second converter error: Currency Rates Source Not Ready | money: TRL 2000000  date: 2015-05-15 00:00:00
First converter error: DOP is not a supported currency | money: DOP 136425999  date: 2018-02-23 00:00:00
Second converter error: Currency Rates Source Not Ready | money: DOP 136425999  date: 2018-02-23 00:00:00
First converter error: TWD is not a supported currency | money: TWD 700000000  date: 2011-09-30 00:00:00
Second converter error: Currency Rates Source Not Ready | money: TWD 700000000  date: 2011-09-30 00:00:00
First converter error: RUR is not a supported currency | money: RUR 65  date: 2014-11-02 00:00:00
Second converter error: Cu

Second converter error: Currency Rates Source Not Ready | money: PKR 120000000  date: 2017-02-10 00:00:00
First converter error: DOP is not a supported currency | money: DOP 45000000  date: 2017-02-26 00:00:00
Second converter error: Currency Rates Source Not Ready | money: DOP 45000000  date: 2017-02-26 00:00:00
First converter error: NGN is not a supported currency | money: NGN 7500000  date: 2015-03-06 00:00:00
Second converter error: Currency Rates Source Not Ready | money: NGN 7500000  date: 2015-03-06 00:00:00
First converter error: DOP is not a supported currency | money: DOP 115126396  date: 2018-02-22 00:00:00
Second converter error: Currency Rates Source Not Ready | money: DOP 115126396  date: 2018-02-22 00:00:00
First converter error: 1997-01-29 not in INR bounds 2009-01-02/2020-06-09 | money: INR 1000000  date: 1997-01-29 00:00:00
First converter error: 2008-10-02 not in INR bounds 2009-01-02/2020-06-09 | money: INR 20000000  date: 2008-10-02 00:00:00
First converter error:

Second converter error: Currency Rates Source Not Ready | money: RUR 35000000  date: 2016-11-10 00:00:00
First converter error: RUR is not a supported currency | money: RUR 400000000  date: 2017-10-05 00:00:00
Second converter error: Currency Rates Source Not Ready | money: RUR 400000000  date: 2017-10-05 00:00:00
First converter error: RUR is not a supported currency | money: RUR 162000000  date: 2017-02-22 00:00:00
Second converter error: Currency Rates Source Not Ready | money: RUR 162000000  date: 2017-02-22 00:00:00
First converter error: RUR is not a supported currency | money: RUR 50000000  date: 2017-08-31 00:00:00
Second converter error: Currency Rates Source Not Ready | money: RUR 50000000  date: 2017-08-31 00:00:00
First converter error: IRR is not a supported currency | money: IRR 2000000000  date: 2010-10-06 00:00:00
Second converter error: Currency Rates Source Not Ready | money: IRR 2000000000  date: 2010-10-06 00:00:00
First converter error: BDT is not a supported curre

Second converter error: Currency Rates Source Not Ready | money: RUR 140000000  date: 2018-04-26 00:00:00
First converter error: 2018-06-08 not in TRL bounds 1999-01-04/2004-12-31 | money: TRL 1500000  date: 2018-06-08 00:00:00
Second converter error: Currency Rates Source Not Ready | money: TRL 1500000  date: 2018-06-08 00:00:00
First converter error: 2018-05-25 not in TRL bounds 1999-01-04/2004-12-31 | money: TRL 100  date: 2018-05-25 00:00:00
Second converter error: Currency Rates Source Not Ready | money: TRL 100  date: 2018-05-25 00:00:00
First converter error: 2016-12-30 not in TRL bounds 1999-01-04/2004-12-31 | money: TRL 600000  date: 2016-12-30 00:00:00
Second converter error: Currency Rates Source Not Ready | money: TRL 600000  date: 2016-12-30 00:00:00
First converter error: 2018-05-18 not in TRL bounds 1999-01-04/2004-12-31 | money: TRL 100  date: 2018-05-18 00:00:00
Second converter error: Currency Rates Source Not Ready | money: TRL 100  date: 2018-05-18 00:00:00
First co

In [20]:
movies_1['revenue_usd'] = converted_revenues
movies_1['budget_usd'] = converted_budget

In [21]:
# set the budget and revenue to NaN for 0's in original dataframe since it is needed for the next update operation.
# (pandas dataframe update operation is filling the NaN values in the original dataframe)
movies['revenue'] = movies['revenue'].replace(0, np.nan)
movies['budget'] = movies['budget'].replace(0, np.nan)

# align imdb_id as index for both dataframe
budget_revenue_df = movies_1.drop(['revenue', 'budget'], axis=1)  \
                    .rename({'revenue_usd': 'revenue', 'budget_usd': 'budget'}, axis=1) \
                    .set_index('imdb_id')
movies_imdb_aligned = movies.set_index('imdb_id')

# update the original dataframe for NaN values
# overwrite false means we will only overwrite NaN values not the normal values for original df
movies_imdb_aligned.update(budget_revenue_df, overwrite=False)

# revert back to integer index and set back to 0 for revenue and budget for NaN values
movies = movies_imdb_aligned.reset_index().drop('id', axis=1)
movies['revenue'] = movies['revenue'].replace(np.nan, 0)
movies['budget'] = movies['budget'].replace(np.nan, 0)
movies = movies.convert_dtypes()

In [22]:
movies

,imdb_id,description,keywords,collection,production_countries,popularity,average_vote,num_votes,language,poster_url,...,title,director,production_companies,cast,genres,revenue,budget,description2,tagline,movie_classification
0,tt0094675,Taisto Kasurinen is a Finnish coal miner whose...,"underdog, prison, factory worker, prisoner, he...",<NA>,Finland,3.860491,7.1,44,fi,/gZCJZOn4l0Zj5hAxsMbxoS6CL0u.jpg,...,Ariel,Aki Kaurismäki,"Villealfa Filmproduction Oy, Finnish Film Foun...","Turo Pajala, Susanna Haavisto, Matti Pellonpää...","drama, crime",0,0,A Finnish man goes to the city to find a job a...,<NA>,unclassified
1,tt0092149,"An episode in the life of Nikander, a garbage ...","salesclerk, helsinki, garbage, independent film",<NA>,Finland,2.292110,7.1,35,fi,/7ad4iku8cYBuB08g9yAU7tHJik5.jpg,...,Shadows in Paradise,Aki Kaurismäki,Villealfa Filmproduction Oy,"Matti Pellonpää, Kati Outinen, Sakari Kuosmane...","drama, comedy",0,0,"An episode in the life of Nikander, a garbage ...",<NA>,unclassified
2,tt0113101,It's Ted the Bellhop's first night on the job....,"hotel, new year's eve, witch, bet, hotel room,...",<NA>,United States of America,9.026586,6.5,539,en,/eQs5hh9rxrk1m4xHsIz1w11Ngqb.jpg,...,Four Rooms,"Allison Anders, Alexandre Rockwell, Robert Rod...","Miramax Films, A Band Apart","Tim Roth, Antonio Banderas, Jennifer Beals, Ma...","crime, comedy",4257354,4000000,Four interlocking tales that take place in a f...,Twelve outrageous guests. Four scandalous requ...,minor success
3,tt0107286,"While racing to a boxing match, Frank, Mike, J...","chicago, drug dealer, boxing match, escape, on...",<NA>,"Japan, United States of America",5.538671,6.4,79,en,/lNXmgUrP6h1nD53gkFh4WDzT6RZ.jpg,...,Judgment Night,Stephen Hopkins,"Universal Pictures, Largo Entertainment, JVC E...","Emilio Estevez, Cuba Gooding Jr., Denis Leary,...","action, thriller, crime",12526677,21000000,Four friends on their way to a boxing match ge...,Don't move. Don't whisper. Don't even breathe.,flop
4,tt0076759,Princess Leia is captured and held hostage by ...,"android, galaxy, hermit, death star, lightsabe...",Star Wars Collection,United States of America,42.149697,8.1,6778,en,/btTdmkgIvOi0FFip1sPuZI2oQG6.jpg,...,Star Wars: Episode IV - A New Hope,George Lucas,"Lucasfilm, Twentieth Century Fox Film Corporation","Mark Hamill, Harrison Ford, Carrie Fisher, Pet...","adventure, action, science fiction",775512064,11000000,Luke Skywalker joins forces with a Jedi Knight...,"A long time ago in a galaxy far, far away...",super hit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45406,tt5943940,A horror comedy spoofing conspiracy theory mov...,<NA>,<NA>,United Kingdom,0.281008,0.0,0,en,/6wr5AWNoyXM08Q3NoQXJ3JWCuYP.jpg,...,Abduction,"Maurice Smith, Mol Smith",<NA>,"Karolina Antosik, Amelie Leroy, Tessa McGinn, ...","fantasy, drama",0,0,A horror comedy spoofing conspiracy theory mov...,Horrifically Funny,unclassified
45407,tt0507700,Fifteen-year-old girl Dotty Fisher is assaulte...,<NA>,<NA>,United States of America,0.001189,0.0,0,en,<NA>,...,Tragedy in a Temporary Town,Sidney Lumet,Showcase Productions,"Lloyd Bridges, Jack Warden, Rafael Campos, Rob...",drama,0,0,Fifteen-year-old girl Dotty Fisher is assaulte...,<NA>,unclassified
45408,tt0133202,"In the 1910s, beautiful young Silja loses both...",<NA>,<NA>,Finland,0.001202,0.0,0,fi,/q9mcHEDpPphN3086oCYiggGkdo.jpg,...,Silja - nuorena nukkunut,Jack Witikka,<NA>,"Heidi Krohn, Jussi Jurkka, Aku Korhonen, Pentt...","drama, romance",0,0,"In the 1910s, beautiful young Silja loses both...",<NA>,unclassified
45409,tt5742932,There's this hot ice-hockey guy called Lauri M...,"fantasy, youth, weird",<NA>,Finland,0.347806,8.0,1,fi,/rKOpJuwb7pTqYVShHM2tl25VxyF.jpg,...,Thick Lashes of Lauri Mäntyvaara,Hannaleena Hauru,Elokuvayhtiö Oy Aamu,"Inka Haapamäki, Rosa Honkonen, Tiitus Rantala,...","romance, comedy",42832,1254040,The love rebels are ready for battle. Satu and...,<NA>,box office bomb


In [23]:
movies.dtypes

imdb_id                         string
description                     string
keywords                        string
collection                      string
production_countries            string
popularity                     float64
average_vote                   float64
num_votes                        Int64
language                        string
poster_url                      string
runtime                          Int64
date                    datetime64[ns]
title                           string
director                        string
production_companies            string
cast                            string
genres                          string
revenue                          Int64
budget                           Int64
description2                    string
tagline                         string
movie_classification            string
dtype: object

In [24]:
# add target column
preprocessing.create_target_feature(movies)

In [25]:
preprocessing.print_stats(movies)

Lenght: 45411 Available budget: 15078 Available revenue: 20339
Missing values (NaN): 

 imdb_id                     0
description               281
keywords                14325
collection              40107
production_countries     2350
popularity                  3
average_vote                3
num_votes                   3
language                    0
poster_url                380
runtime                     1
date                       27
title                       0
director                  119
production_companies     3830
cast                      652
genres                      9
revenue                     0
budget                      0
description2              709
tagline                 23304
movie_classification        0
dtype: int64

Usable rows according to target: 10562


In [26]:
movies.to_csv(f"{DATA_FOLDER}/final_dataset_enriched.csv", index=False, header=True)

## Second dataset

Second dataset URL: https://www.kaggle.com/danielgrijalvas/movies

In [27]:
movies = pd.read_csv(f'{DATA_FOLDER}/final_dataset_enriched.csv', parse_dates=['date']).convert_dtypes()
movies_2 = pd.read_csv(f'{DATA_FOLDER}/datasets_2745_4700_movies.csv', parse_dates=['released'], encoding = "ISO-8859-1").convert_dtypes()
movies_2.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
0,8000000,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414,Stand by Me,R,1986-08-22,89,8.1,Wil Wheaton,299174,Stephen King,1986
1,6000000,Paramount Pictures,USA,John Hughes,Comedy,70136369,Ferris Bueller's Day Off,PG-13,1986-06-11,103,7.8,Matthew Broderick,264740,John Hughes,1986
2,15000000,Paramount Pictures,USA,Tony Scott,Action,179800601,Top Gun,PG,1986-05-16,110,6.9,Tom Cruise,236909,Jim Cash,1986
3,18500000,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248,Aliens,R,1986-07-18,137,8.4,Sigourney Weaver,540152,James Cameron,1986
4,9000000,Walt Disney Pictures,USA,Randal Kleiser,Adventure,18564613,Flight of the Navigator,PG,1986-08-01,90,6.9,Joey Cramer,36636,Mark H. Baker,1986


In [28]:
movies_2.dtypes

budget               Int64
company             string
country             string
director            string
genre               string
gross                Int64
name                string
rating              string
released    datetime64[ns]
runtime              Int64
score              float64
star                string
votes                Int64
writer              string
year                 Int64
dtype: object

In [29]:
preprocessing.print_stats(movies_2, revenue_col='gross')

Lenght: 6820 Available budget: 4638 Available revenue: 6820
Missing values (NaN): 

 budget      0
company     0
country     0
director    0
genre       0
gross       0
name        0
rating      0
released    0
runtime     0
score       0
star        0
votes       0
writer      0
year        0
dtype: int64


Since we do not have imdb_id for this dataset, we will use the movie title and the date of the movie for matching. After matching, we can use imdb_ids for update operation.

In [30]:
# keep only the rows matching between two dfs
movies_2 = movies[['imdb_id', 'title', 'date']].merge(movies_2, left_on=['title', 'date'], 
                                                    right_on=['name', 'released'], how='inner')
movies_2 = movies_2.drop(['rating', 'score', 'votes', 'writer', 'year', 'name', 'released'], axis=1)  \
                    .rename({'gross': 'revenue', 'company': 'production_companies', 
                             'country': 'production_countries', 
                             'genre': 'genres', 'star':'cast'}, axis=1)

In [32]:
budget_none = movies[((movies.budget == 0) & (movies.revenue != 0))]
revenue_none = movies[((movies.budget != 0) & (movies.revenue == 0))]

budget_exist = movies_2[((movies_2.budget != 0))]
revenue_exist = movies_2[(movies_2.revenue != 0)]

budget_movies = budget_none.merge(budget_exist, on=['imdb_id'], how='inner')
revenue_movies = revenue_none.merge(revenue_exist, on=['imdb_id'], how='inner')

In [33]:
print(f'Budget exist on second df for NaN budget values: {len(budget_movies)}\n'
      f'Revenue exist on second df for NaN rev. values: {len(revenue_movies)}')

Budget exist on second df for NaN budget values: 0
Revenue exist on second df for NaN rev. values: 11


we have around 11 movies which revenue info exist on second df. The low number of information available is normal because we have already exploited the info from IMDB on the first dataframe.

In [34]:
movies['revenue'] = movies['revenue'].replace(0, np.nan)
movies['budget'] = movies['budget'].replace(0, np.nan)

# align imdb_id as index for both dataframe
movies_2_imdb_aligned = movies_2.set_index('imdb_id')
movies_imdb_aligned = movies.set_index('imdb_id')

# update the original dataframe for NaN values
# overwrite false means we will only overwrite NaN values not the normal values for original df
movies_imdb_aligned.update(movies_2_imdb_aligned, overwrite=False)

# revert back to integer index and set back to 0 for revenue and budget for NaN values
movies = movies_imdb_aligned.reset_index()
movies['revenue'] = movies['revenue'].replace(np.nan, 0)
movies['budget'] = movies['budget'].replace(np.nan, 0)
movies = movies.convert_dtypes()

In [35]:
# add target column
preprocessing.create_target_feature(movies)

In [36]:
preprocessing.print_stats(movies)

Lenght: 45411 Available budget: 15078 Available revenue: 20368
Missing values (NaN): 

 imdb_id                     0
description               281
keywords                14325
collection              40107
production_countries     2350
popularity                  3
average_vote                3
num_votes                   3
language                    0
poster_url                380
runtime                     1
date                       27
title                       0
director                  119
production_companies     3830
cast                      652
genres                      9
revenue                     0
budget                      0
description2              709
tagline                 23304
movie_classification        0
dtype: int64

Usable rows according to target: 10573


In [37]:
movies.to_csv(f"{DATA_FOLDER}/final_dataset_enriched.csv", index=False, header=True)

## Third dataset

The third dataset URL: https://www.kaggle.com/stephanerappeneau/350-000-movies-from-themoviedborg

In [38]:
movies = pd.read_csv(f'{DATA_FOLDER}/final_dataset_enriched.csv', parse_dates=['date']).convert_dtypes()
movies_3 = df = pd.read_csv(f"{DATA_FOLDER}/AllMoviesDetailsCleaned.csv", encoding='utf-8-sig', sep=";", 
                 engine="python",parse_dates=["release_date"]).convert_dtypes()
movies_3.head()

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,production_companies_number,production_countries_number,spoken_languages_number
0,2,0,Drama|Crime,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,0.823904,Villealfa Filmproduction Oy,Finland,...,69,suomi,Released,<NA>,Ariel,7.1,40,2,1,2
1,3,0,Drama|Comedy,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",0.47445,Villealfa Filmproduction Oy,Finland,...,76,English,Released,<NA>,Shadows in Paradise,7.0,32,1,1,3
2,5,4000000,Crime|Comedy,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,1.698,Miramax Films,United States of America,...,98,English,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,485,2,1,1
3,6,0,Action|Thriller|Crime,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",1.32287,Universal Pictures,Japan,...,110,English,Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,6.5,69,3,2,1
4,8,42000,Documentary,tt0825671,en,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,0.054716,inLoops,Austria,...,80,English,Released,A Megacities remix.,Life in Loops (A Megacities RMX),6.4,4,1,1,5


In [39]:
movies_3.isna().sum()

id                                  0
budget                              0
genres                         121529
imdb_id                         78480
original_language                 203
original_title                      1
overview                        50984
popularity                          0
production_companies                0
production_countries                0
release_date                    24046
revenue                             0
runtime                         36792
spoken_languages                 4460
status                           1988
tagline                        276113
title                               1
vote_average                        0
vote_count                          0
production_companies_number         0
production_countries_number         0
spoken_languages_number             0
dtype: int64

In [40]:
preprocessing.print_stats(movies_3)

Lenght: 329044 Available budget: 15781 Available revenue: 8690
Missing values (NaN): 

 id                                  0
budget                              0
genres                         121529
imdb_id                         78480
original_language                 203
original_title                      1
overview                        50984
popularity                          0
production_companies                0
production_countries                0
release_date                    24046
revenue                             0
runtime                         36792
spoken_languages                 4460
status                           1988
tagline                        276113
title                               1
vote_average                        0
vote_count                          0
production_companies_number         0
production_countries_number         0
spoken_languages_number             0
dtype: int64


In [41]:
# first convert dataframe with the same format
movies_3 = movies_3.drop(['id', 'production_companies_number', 'spoken_languages_number', 
                          'production_companies_number', 'production_countries_number',
                          'status', 'spoken_languages', 'original_title'], 1)
movies_3 = movies_3.rename(columns={'original_language': 'language', 
                                    'release_date': 'date', 'overview': 'description',
                                   'vote_average':'average_vote', 'vote_count': 'num_votes'})

We have to convert | seperated genres to comma seperated ones.

In [42]:
movies_3['genres'] = movies_3['genres'].str.split('|').str.join(', ')

Keep only the movies with valid imdb_ids

In [43]:
imdb_id_valid = movies_3[~movies_3.imdb_id.isna()]

In [44]:
imdb_id_valid

,budget,genres,imdb_id,language,description,popularity,production_companies,production_countries,date,revenue,runtime,tagline,title,average_vote,num_votes
0,0,"Drama, Crime",tt0094675,fi,Taisto Kasurinen is a Finnish coal miner whose...,0.823904,Villealfa Filmproduction Oy,Finland,1988-10-21,0,69,<NA>,Ariel,7.1,40
1,0,"Drama, Comedy",tt0092149,fi,"An episode in the life of Nikander, a garbage ...",0.47445,Villealfa Filmproduction Oy,Finland,1986-10-16,0,76,<NA>,Shadows in Paradise,7.0,32
2,4000000,"Crime, Comedy",tt0113101,en,It's Ted the Bellhop's first night on the job....,1.698,Miramax Films,United States of America,1995-12-25,4300000,98,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,485
3,0,"Action, Thriller, Crime",tt0107286,en,"While racing to a boxing match, Frank, Mike, J...",1.32287,Universal Pictures,Japan,1993-10-15,12136938,110,Don't move. Don't whisper. Don't even breathe.,Judgment Night,6.5,69
4,42000,Documentary,tt0825671,en,Timo Novotny labels his new project an experim...,0.054716,inLoops,Austria,2006-01-01,0,80,A Megacities remix.,Life in Loops (A Megacities RMX),6.4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329034,0,Drama,tt0106530,es,no overview,0.00015,Origen Producciones Cinematograficas S.A.,Spain,1994-04-29,0,90,<NA>,Wild Boys,0.0,0
329035,0,Science Fiction,tt6451666,ja,"Aki isn’t a brave little boy, but he’s dedicat...",0.01455,Kamikaze Douga,Japan,2017-07-02,0,46,<NA>,Cocolors,0.0,0
329039,0,"Documentary, History, TV Movie",tt5430190,en,The story of Hitler’s final hours told by peop...,0.0045,History Channel,United States of America,2016-01-21,0,46,"April 30th, 1945",The Day Hitler Died,7.0,1
329042,0,"Drama, Comedy",tt6391664,en,Director Levi A. Taylor invites you on a nosta...,0.0096,none,United States of America,2016-10-11,0,12,<NA>,Conspiracy P.I.E,0.0,0


In [45]:
movies['revenue'] = movies['revenue'].replace(0, np.nan)
movies['budget'] = movies['budget'].replace(0, np.nan)

# align imdb_id as index for both dataframe
imdb_id_valid = imdb_id_valid.set_index('imdb_id')
movies_imdb_aligned = movies.set_index('imdb_id')

# update the original dataframe for NaN values
# overwrite false means we will only overwrite NaN values not the normal values for original df
movies_imdb_aligned.update(imdb_id_valid, overwrite=False)

# revert back to integer index and set back to 0 for revenue and budget for NaN values
movies = movies_imdb_aligned.reset_index()
movies['revenue'] = movies['revenue'].replace(np.nan, 0)
movies['budget'] = movies['budget'].replace(np.nan, 0)
movies = movies.convert_dtypes()

In [46]:
# add target column
preprocessing.create_target_feature(movies)

In [47]:
preprocessing.print_stats(movies)

Lenght: 45411 Available budget: 15082 Available revenue: 20369
Missing values (NaN): 

 imdb_id                     0
description               281
keywords                14325
collection              40107
production_countries       31
popularity                  3
average_vote                3
num_votes                   3
language                    0
poster_url                380
runtime                     1
date                       26
title                       0
director                  119
production_companies       40
cast                      652
genres                      9
revenue                     0
budget                      0
description2              709
tagline                 23280
movie_classification        0
dtype: int64

Usable rows according to target: 10577


In [48]:
movies.to_csv(f"{DATA_FOLDER}/final_dataset_enriched.csv", index=False, header=True)